# Domain Specific Language (DSL) Showcase
## Contents
1. [Catalyst.jl](https://github.com/SciML/Catalyst.jl/tree/master) (Existing Biochemical Reaction DSL) 
2. Custom DSL for Project

## 1. Catalyst.jl
We will now explore Catalyst.jl for specifiying biochemical reactions in human-readable format for conversion into an Ordinary Differential Equation (ODE) system.

In [ ]:
# Import Catalyst Package, Latexify (For viewing reactions)
using Catalyst, Latexify

## Review of original transcription and translation reactions:
If you recall our simple framework of reactions for transcription and translation are:

*Transcription*:
\begin{align*}
    g_i + \pi &\xrightleftharpoons[k_{1;i}^{-}]{k_{1;i}^{+}} X_i \\
    X_i &\xrightarrow{k^{\theta}_{1;i}} g_i + \pi + m_i \\
    m_i &\xrightarrow{\delta_i} \varnothing  
\end{align*}

*Translation*:
\begin{align*}
    m_i + \rho &\xrightleftharpoons[k_{2;i}^{-}]{k_{2;i}^{+}} \Xi_i \\
    \Xi_i &\xrightarrow{k^{\theta}_{2;i}} m_i + \rho + x_i \\
    x^{\omega}_i &\xrightarrow{\lambda} \varnothing  
\end{align*}

Let's look at what it looks like with the Catalyst.jl DSL!

In [ ]:

node_1 = @reaction_network begin
    (k1⁺₁, k1⁻₁), g₁ + p <--> X₁ # Gene 1 binds with RNA polymerase to form the intermediate complex in a reversible reaction
    (k1θ₁), Xᵢ --> g₁ + p + m₁ # The intermediate complex reforms gene 1 and RNAP after transcription and mRNA 1 is formed
    (δ₁), m₁ --> ∅ # mRNA 1 decays at a rate δ₁
    (k2⁺₁, k2⁻₁), m₁ + r <--> Ξ₁ # mRNA 1 binds with a ribosome to form the intermediate complex in a reversible reaction
    (k2θ₁), Ξ₁ --> m₁ + r + x₁ # The intermediate complex reforms mRNA 1 and the ribosome after translation and protein 1 is formed
    (λ), x₁ --> ∅ # Protein 1 decays at a rate λ
end

latexify(node_1)

Look how close it looks to the original reactions! But if we want to apply our resource conservation framework, we'll have to implement new code to generate the ODE system and code a new DSL.

## 2. Custom DSL For Project
To implement the mathematical model in this project, a whole back-end "engine" was developed to generate the ODE system. A DSL front-end was also made to allow you to easily enter the original reactions. Let's start with the front-end.

In [3]:
# Import necessary files
include("DSL_backend/dsl.jl")


@TFinactivate (macro with 1 method)

Now let's create a node! You can see that a different macro (@createnode) is used compared to Catalyst.jl. Also, another feature is that you can specify initial conditions very easily when you create a node unlike Catalyst! The reactions specified in the DSL are processed into a custom data structure (Node type) in the back-end.

In [2]:
# Create Node
node_1 = @createnode begin
    A ~ g = 0.5 # Node A with initial gene concentration of 0.5
    [k1⁺₁ = 4, k1⁻₁ = 1], g₁ + p <--> C₁ # gene (g₁) binds with polymerase (p) to create the intermediate complex C₁
    [k1θ₁ = 5], C --> g₁ + p + m₁ # the intermediate complex reforms the gene and polymerase while giving mRNA (m₁) after transcription
    [k2⁺₁ = 4, k2⁻₁ = 1], m₁ + r <--> X₁ # m₁ binds with a ribosome (r) to create the intermediate complex X₁
    [k2θ₁ = 5], X₁ --> m₁ + r + x₁ # The intermediate complex gives the protein (x₁) while returning the gene and ribosome
    [d₁ = 0.04], m₁ --> ∅ # mRNA decays at a rate of d = 0.04, protein dilution rate is dealt with later
end

Node(:A, 4.0, 1.0, 5.0, 4.0, 1.0, 5.0, 0.04, 0.5)

Now to specify the edges. There are a few types: activation (@activate), repression (@repress), a combination of the two (@combi), TF activation (@TFactivate), TF inactivation (@TFinactivate) and annihilation (@annihilate). 

When creating these, you can conveniently specify which the nodes that these edges connect. (e.g. A -> B means that node A activates node B)

A key feature is to the notation to specify the relations between nodes with an arrow notation to conveniently understand your system after you create it! ("->" is activation, "-!" is repression/inactivation and "-X-" is annihilation).

In [8]:
# Activation
edge_1 = @activate begin
    A -> B # Node A activates Node B
    [k'⁺₂ = 100, k'⁻₂ = 1], g₂ + 3x₁ <--> c₂ # gene (g₂) + 3x₁ (TF) gives activated complex (c₂ )
end

# Repression
edge_2 = @repress begin
    B -! A # Node B represses Node A
    [k'⁺₂ = 100, k'⁻₂ = 1], g₁ + 3x₂ <--> c⁰₁ # gene (g₁) + 3x₂ (TF) gives repressed complex (c⁰₁) 
end

# Combinatorial promoter (Simplifies notation for specifically specifying activation and repression)
edges_3 = @combi begin
    A -> C, B -! C  # A activates C and B represses C
    [k⁺ₐ₃ = 100, k⁻ₐ₃ = 1], g₃ + x₁ <--> c₃ # Activation reaction
    [k⁺ᵣ₃ = 100, k⁻ᵣ₃ = 1], g₃ + 3x₂ <--> c⁰₃ # Repression reaction
end
# Note that this produces 2 edges, one for activation and one for repression

# Annihilation
edge_4 = @annihilate begin
    A -X- B # A and B annihilate each other 
    [k⁺₁₂ = 1000, k⁻₁₂ = 0.1], m₁ + m₂ --> m∅ # In this case it is mRNA annihilation (you must use m in the variable)
end
# Using any other symbol without "m" is detected as protein annihilation

# TF activation
edge_5 = @TFactivate begin
    I₁ -> A ~ I₁ = 30 # The TF produced by A is activated by external inducer I₁. The initial value of I₁ is 30.
    [κ⁺₁ = 1000, κ⁺₂ = 1], x⁰₁ + 2I₁ <--> x⁺₁ # Inactive TF is activated by I₁ with a cooperativity of 2
end

# TF inactivation
edge_5 = @TFinactivate begin
    I₂ -! A ~ I₂ = 30 # The TF produced by A is inactivated by inducer I₂
    [κ⁺₁ = 1000, κ⁺₂ = 1], x⁺₁ + 2I₂ <--> x⁰₁ # Active TF is inactivated by I₂ with a cooperatvity of 2
end

ProtoIndEdge(2, :I₂, :A, 1000.0, 1.0, 2, 30)

### DSL Showcase (Actual Use)
Now that we've seen the showcase of all the macros, we can now see how you would actually use it. 

One thing to note is that the symbols for nodes are specified in the edges by they do not mean anything yet.

To link the symbols to nodes, we must create the nodes first. Now let's create a system of 3 nodes.

In [ ]:
# DSL Showcase (Actual Use)
node_1 = @createnode begin
    A ~ g = 0.5
    [k1⁺₁ = 4, k1⁻₁ = 1], g₁ + p <--> C₁
    [k1θ₁ = 5], C --> g₁ + p + m₁
    [k2⁺₁ = 4, k2⁻₁ = 1], m₁ + r <--> X₁
    [k2θ₁ = 5], X₁ --> m₁ + r + x₁
    [d₁ = 0.04], m₁ --> ∅
end

node_2 = @createnode begin
    B ~ g = 0.5
    [k1⁺₂ = 4, k1⁻₂ = 1], g₂ + p <--> C₂
    [k1θ₂ = 5], C₂ --> g₂ + p + m₂
    [k2⁺₂ = 4, k2⁻₂ = 1], m₂ + r <--> X₂
    [k2θ₂ = 5], X₂ --> m₂ + r + x₂
    [d₂ = 0.04], m₂ --> ∅
end

node_3 = @createnode begin
    C ~ g = 0.5
    [k1⁺₃ = 4, k1⁻₃ = 1], g₃ + p <--> C₃
    [k1θ₃ = 5], C₃ --> g₃ + p + m₃
    [k2⁺₃ = 4, k2⁻₃ = 1], m₃ + r <--> X₃
    [k2θ₃ = 5], X₃ --> m₃ + r + x₃
    [d₃ = 0.04], m₃ --> ∅
end

# You must use the compilenodes() function to compile all the nodes into a variable
nodes = compilenodes(node_1, node_2, node_3)

As you can see, you have to first create the nodes that have symbols as names (A, B and C) in this case. Nodes then have to be combined using the "compilenodes()" function. 

Now when we create edges, we need to use the "edge(nodes, macro)" or "edges(nodes, macro)" functions to connect the edge to the nodes:

In [ ]:
# Note the use of the edge function which contains both the vector of nodes specified and the macros
edge1 = edge(nodes, @repress begin 
    A -! B
    [k'⁺₂ = 100, k'⁻₂ = 1], g₂ + 3x₁ <--> c₂
end)

edge2_3 =  edges(nodes, @combi begin
    A -> C, B -! C
    [k⁺ₐ₃ = 100, k⁻ₐ₃ = 1], g₃ + x₁ <--> c₃
    [k⁺ᵣ₃ = 99, k⁻ᵣ₃ = 2], g₃ + 3x₂ <--> c⁰₃
end)

edge4 = edge(nodes, @annhilate begin
    A -X- B
    [k⁺₁₂ = 1000, k⁻₁₂ = 0.1], m₁ + m₂ --> m∅
end)

edge5 = edge(nodes, @TFinactivate begin
    I₁ -! A ~ I₁ = 30
    [κ⁺₁ = 1000, κ⁺₂ = 1], x⁰₁ + 2I₁ <--> x⁺₁
end)

And with this, we can create some systems!